In [ ]:
from flask import Flask
import requests
from bs4 import BeautifulSoup
import mysql.connector

app = Flask(__name__)

# MySQL configuration
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'RootPass@#007',
    'database': 'price_tracker_db'
}

# Connect to MySQL
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

# Create a table to store product data if not exists
create_table_query = """
CREATE TABLE IF NOT EXISTS products (
    id INT AUTO_INCREMENT PRIMARY KEY,
    product_name VARCHAR(255),
    price FLOAT,
    source_url VARCHAR(255)
)
"""
cursor.execute(create_table_query)

# List of product URLs
product_urls = {
    'AirPods Max': [
        'https://www.amazon.com/New-Apple-AirPods-Max-Space/dp/B08PZHYWJS',
        'https://www.walmart.com/ip/Apple-AirPods-Max-Space-Gray/509451094',
        'https://www.bestbuy.com/site/apple-airpods-max-space-gray/6373460.p?skuId=6373460'
    ],

    'Ring Video Doorbell': [
        'https://www.amazon.com/ring-doorbell/s?k=ring+doorbell',
        'https://www.walmart.com/c/kp/ring-video-doorbells',
        'https://www.bestbuy.com/site/ring/ring-smart-doorbells/pcmcat748301722121.c?id=pcmcat748301722121'
    ],

    'Instant Pot Pro': [
        'https://www.amazon.com/Instant-Pot-Pressure-Steamer-Sterilizer/dp/B08PQ2KWHS',
        'https://www.walmart.com/ip/Instant-Pot-Pro-6-Quart-Electric-Pressure-Cooker-10-in-1-Slow-Rice-Grain-Steamer-Saut-Sous-Vide-Yogurt-Maker-Sterilizer-Warmer/674041054?from=/search',
        'https://www.bestbuy.com/site/instant-pot-6qt-pro-electric-pressure-cooker-black/6452130.p?skuId=6452130'
    ],

    'Dyson V8 Absolute': [
        'https://www.amazon.com/Dyson-V8-Absolute-Cordless-Vacuum/dp/B01EX9D3UW',
        'https://www.walmart.com/ip/Dyson-V8-Absolute-Cordless-Vacuum-First-Generation-Silver-New/1068750176',
        'https://www.bestbuy.com/site/dyson-v8-cordless-vacuum-silver-nickel/6500879.p?skuId=6500879'
    ],

    'Meta Quest 3': [
        'https://www.amazon.com/Meta-Quest-128GB-Breakthrough-Reality-3/dp/B0C8VKH1ZH',
        'https://www.walmart.com/ip/Meta-Quest-3-128GB-Breakthrough-Mixed-Reality-Powerful-Performance-Asgard-s-Wrath-2-Bundle/2212742363?adsRedirect=true',
        'https://www.bestbuy.com/site/meta-quest-3-breakthrough-mixed-reality-128gb-white/6549064.p?skuId=6549064'
    ],
}

# Function to scrape product data and store it in the database
def scrape_and_store_data(product_name, urls):
    for url in urls:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract product name and price based on the website structure
        if 'amazon' in url:
            buybox_element = soup.find('div', {'id': 'buybox'})
            if buybox_element:
                price_element = buybox_element.find('span', {'class': 'a-offscreen'})
                if price_element:
                    price = float(price_element.get_text().replace('$', '').replace(',', '').strip())
                    product_name_element = soup.find('span', {'id': 'productTitle'})
                    product_name = product_name_element.get_text().strip()
                    website = 'amazon'

                    # Insert data into the database
                    insert_query = "INSERT INTO products (product_name, price, source_url, website) VALUES (%s, %s, %s, %s)"
                    cursor.execute(insert_query, (product_name, price, url))
                    conn.commit()
        elif 'walmart' in url:
            price_element = soup.find('span', {'itemprop': 'price', 'aria-hidden': 'false'})
            if price_element:
                price = float(price_element.get_text().replace('$', '').replace(',', '').strip())
                product_name_element = soup.find('h1', {'class': 'prod-ProductTitle'})
                product_name = product_name_element.get_text().strip()
                website = 'walmart'
                # Insert data into the database
                insert_query = "INSERT INTO products (product_name, price, source_url, website) VALUES (%s, %s, %s, %s)"
                cursor.execute(insert_query, (product_name, price, url))
                conn.commit()
        elif 'bestbuy' in url:
            price_element = soup.find('div', {'class': 'priceView-hero-price'})
            if price_element:
                price = float(price_element.find('span', {'aria-hidden': 'true'}).get_text().replace('$', '').replace(',', '').strip())
                product_name_element = soup.find('h1', {'class': 'sku-title'})
                product_name = product_name_element.get_text().strip()
                website = 'bestbuy'

                # Insert data into the database
                insert_query = "INSERT INTO products (product_name, price, source_url, website) VALUES (%s, %s, %s, %s)"
                cursor.execute(insert_query, (product_name, price, url))
                conn.commit()
        else:
            product_name_element = None
            price_element = None

        if product_name_element and price_element:
            product_name = product_name_element.get_text().strip()
            price = float(price_element.get_text().replace('$', '').replace(',', '').strip())
            website = 'None'
            # Insert data into the database
            insert_query = "INSERT INTO products (product_name, price, source_url, website) VALUES (%s, %s, %s, %s)"
            cursor.execute(insert_query, (product_name, price, url))
            conn.commit()

# Route to trigger scraping and storing data
@app.route('/')
def scrape_data():
    for product_name, urls in product_urls.items():
        scrape_and_store_data(product_name, urls)
    return 'Data scraped and stored successfully.'

if __name__ == '__main__':
    app.run(debug=True)

Normal


In [2]:
import requests
from bs4 import BeautifulSoup
import mysql.connector

# MySQL configuration
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'RootPass@#007',
    'database': 'price_tracker_db'
}

# Connect to MySQL
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

# Create a table to store product data if not exists
create_table_query = """
CREATE TABLE IF NOT EXISTS # MySQL configuration
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'RootPass@#007',
    'database': 'price_tracker_db'
}
)
"""
cursor.execute(create_table_query)

# List of product URLs
product_urls = {
    'AirPods Max': [
        'https://www.amazon.com/New-Apple-AirPods-Max-Space/dp/B08PZHYWJS',
        'https://www.walmart.com/ip/Apple-AirPods-Max-Space-Gray/509451094',
        'https://www.bestbuy.com/site/apple-airpods-max-space-gray/6373460.p?skuId=6373460'
    ],

    'Ring Video Doorbell': [
        'https://www.amazon.com/ring-doorbell/s?k=ring+doorbell',
        'https://www.walmart.com/c/kp/ring-video-doorbells',
        'https://www.bestbuy.com/site/ring/ring-smart-doorbells/pcmcat748301722121.c?id=pcmcat748301722121'
    ],

    'Instant Pot Pro': [
        'https://www.amazon.com/Instant-Pot-Pressure-Steamer-Sterilizer/dp/B08PQ2KWHS',
        'https://www.walmart.com/ip/Instant-Pot-Pro-6-Quart-Electric-Pressure-Cooker-10-in-1-Slow-Rice-Grain-Steamer-Saut-Sous-Vide-Yogurt-Maker-Sterilizer-Warmer/674041054?from=/search',
        'https://www.bestbuy.com/site/instant-pot-6qt-pro-electric-pressure-cooker-black/6452130.p?skuId=6452130'
    ],

    'Dyson V8 Absolute': [
        'https://www.amazon.com/Dyson-V8-Absolute-Cordless-Vacuum/dp/B01EX9D3UW',
        'https://www.walmart.com/ip/Dyson-V8-Absolute-Cordless-Vacuum-First-Generation-Silver-New/1068750176',
        'https://www.bestbuy.com/site/dyson-v8-cordless-vacuum-silver-nickel/6500879.p?skuId=6500879'
    ],

    'Meta Quest 3': [
        'https://www.amazon.com/Meta-Quest-128GB-Breakthrough-Reality-3/dp/B0C8VKH1ZH',
        'https://www.walmart.com/ip/Meta-Quest-3-128GB-Breakthrough-Mixed-Reality-Powerful-Performance-Asgard-s-Wrath-2-Bundle/2212742363?adsRedirect=true',
        'https://www.bestbuy.com/site/meta-quest-3-breakthrough-mixed-reality-128gb-white/6549064.p?skuId=6549064'
    ],
}

# Function to scrape product data and store it in the database
def scrape_and_store_data(product_name, urls):
    for url in urls:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract product name and price based on the website structure
        if 'amazon' in url:
            buybox_element = soup.find('div', {'id': 'buybox'})
            if buybox_element:
                price_element = buybox_element.find('span', {'class': 'a-offscreen'})
                if price_element:
                    price = float(price_element.get_text().replace('$', '').replace(',', '').strip())
                    product_name_element = soup.find('span', {'id': 'productTitle'})
                    product_name = product_name_element.get_text().strip()
                    website = 'amazon'

                    # Insert data into the database
                    insert_query = "INSERT INTO products (product_name, price, source_url, website) VALUES (%s, %s, %s, %s)"
                    cursor.execute(insert_query, (product_name, price, url, website))
                    conn.commit()
        elif 'walmart' in url:
            price_element = soup.find('span', {'itemprop': 'price', 'aria-hidden': 'false'})
            if price_element:
                price = float(price_element.get_text().replace('$', '').replace(',', '').strip())
                product_name_element = soup.find('h1', {'class': 'prod-ProductTitle'})
                product_name = product_name_element.get_text().strip()
                website = 'walmart'
                # Insert data into the database
                insert_query = "INSERT INTO products (product_name, price, source_url, website) VALUES (%s, %s, %s, %s)"
                cursor.execute(insert_query, (product_name, price, url, website))
                conn.commit()
        elif 'bestbuy' in url:
            price_element = soup.find('div', {'class': 'priceView-hero-price'})
            if price_element:
                price = float(price_element.find('span', {'aria-hidden': 'true'}).get_text().replace('$', '').replace(',', '').strip())
                product_name_element = soup.find('h1', {'class': 'sku-title'})
                product_name = product_name_element.get_text().strip()
                website = 'bestbuy'

                # Insert data into the database
                insert_query = "INSERT INTO products (product_name, price, source_url, website) VALUES (%s, %s, %s, %s)"
                cursor.execute(insert_query, (product_name, price, url, website))
                conn.commit()
        else:
            product_name_element = None
            price_element = None

# Run scraping and storing data
for product_name, urls in product_urls.items():
    scrape_and_store_data(product_name, urls)

# Close the database connection
conn.close()


Try_2


In [9]:
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'RootPass@#007',
    'database': 'price_tracker_db'
}

# Connect to MySQL
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()


cursor.execute(create_table_query)

# List of product URLs
product_urls = {
    'AirPods Max': [
        'https://www.amazon.com/New-Apple-AirPods-Max-Space/dp/B08PZHYWJS',
        'https://www.walmart.com/ip/Apple-AirPods-Max-Space-Gray/509451094',
        'https://www.bestbuy.com/site/apple-airpods-max-space-gray/6373460.p?skuId=6373460'
    ],

    'Ring Video Doorbell': [
        'https://www.amazon.com/ring-doorbell/s?k=ring+doorbell',
        'https://www.walmart.com/c/kp/ring-video-doorbells',
        'https://www.bestbuy.com/site/ring/ring-smart-doorbells/pcmcat748301722121.c?id=pcmcat748301722121'
    ],

    'Instant Pot Pro': [
        'https://www.amazon.com/Instant-Pot-Pressure-Steamer-Sterilizer/dp/B08PQ2KWHS',
        'https://www.walmart.com/ip/Instant-Pot-Pro-6-Quart-Electric-Pressure-Cooker-10-in-1-Slow-Rice-Grain-Steamer-Saut-Sous-Vide-Yogurt-Maker-Sterilizer-Warmer/674041054?from=/search',
        'https://www.bestbuy.com/site/instant-pot-6qt-pro-electric-pressure-cooker-black/6452130.p?skuId=6452130'
    ],

    'Dyson V8 Absolute': [
        'https://www.amazon.com/Dyson-V8-Absolute-Cordless-Vacuum/dp/B01EX9D3UW',
        'https://www.walmart.com/ip/Dyson-V8-Absolute-Cordless-Vacuum-First-Generation-Silver-New/1068750176',
        'https://www.bestbuy.com/site/dyson-v8-cordless-vacuum-silver-nickel/6500879.p?skuId=6500879'
    ],

    'Meta Quest 3': [
        'https://www.amazon.com/Meta-Quest-128GB-Breakthrough-Reality-3/dp/B0C8VKH1ZH',
        'https://www.walmart.com/ip/Meta-Quest-3-128GB-Breakthrough-Mixed-Reality-Powerful-Performance-Asgard-s-Wrath-2-Bundle/2212742363?adsRedirect=true',
        'https://www.bestbuy.com/site/meta-quest-3-breakthrough-mixed-reality-128gb-white/6549064.p?skuId=6549064'
    ],
}



# Function to scrape product data and store it in the database
def scrape_and_store_data(product_name, urls):
    for url in urls:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract product name and price based on the website structure
        if 'amazon' in url:
            buybox_element = soup.find('div', {'id': 'buybox'})
            if buybox_element:
                price_element = buybox_element.find('span', {'class': 'a-offscreen'})
                if price_element:
                    price = float(price_element.get_text().replace('$', '').replace(',', '').strip())
                    product_name_element = soup.find('span', {'id': 'productTitle'})
                    product_name = product_name_element.get_text().strip()
                    website = 'amazon'

                    # Insert data into the database
                    insert_query = "INSERT INTO products (product_name, price, source_url, website) VALUES (%s, %s, %s, %s)"
                    cursor.execute(insert_query, (product_name, price, url, website))
                    conn.commit()
                    print(f"Inserted data for {product_name} from {website}")
                else:
                    print(f"No price element found for {url}")
            else:
                print(f"No buybox element found for {url}")
        elif 'walmart' in url:
            price_element = soup.find('span', {'itemprop': 'price', 'aria-hidden': 'false'})
            if price_element:
                price = float(price_element.get_text().replace('$', '').replace(',', '').strip())
                product_name_element = soup.find('h1', {'class': 'prod-ProductTitle'})
                product_name = product_name_element.get_text().strip()
                website = 'walmart'
                # Insert data into the database
                insert_query = "INSERT INTO products (product_name, price, source_url, website) VALUES (%s, %s, %s, %s)"
                cursor.execute(insert_query, (product_name, price, url, website))
                conn.commit()
                print(f"Inserted data for {product_name} from {website}")
            else:
                print(f"No price element found for {url}")
        elif 'bestbuy' in url:
            price_element = soup.find('div', {'class': 'priceView-hero-price'})
            if price_element:
                price = float(price_element.find('span', {'aria-hidden': 'true'}).get_text().replace('$', '').replace(',', '').strip())
                product_name_element = soup.find('h1', {'class': 'sku-title'})
                product_name = product_name_element.get_text().strip()
                website = 'bestbuy'

                # Insert data into the database
                insert_query = "INSERT INTO products (product_name, price, source_url, website) VALUES (%s, %s, %s, %s)"
                cursor.execute(insert_query, (product_name, price, url, website))
                conn.commit()
                print(f"Inserted data for {product_name} from {website}")
            else:
                print(f"No price element found for {url}")
        else:
            print(f"Unsupported website for {url}")

# Run scraping and storing data
for product_name, urls in product_urls.items():
    scrape_and_store_data(product_name, urls)

# Close the database connection
conn.close()


No buybox element found for https://www.amazon.com/New-Apple-AirPods-Max-Space/dp/B08PZHYWJS
No price element found for https://www.walmart.com/ip/Apple-AirPods-Max-Space-Gray/509451094
No price element found for https://www.bestbuy.com/site/apple-airpods-max-space-gray/6373460.p?skuId=6373460
No buybox element found for https://www.amazon.com/ring-doorbell/s?k=ring+doorbell
No price element found for https://www.walmart.com/c/kp/ring-video-doorbells
No price element found for https://www.bestbuy.com/site/ring/ring-smart-doorbells/pcmcat748301722121.c?id=pcmcat748301722121
No buybox element found for https://www.amazon.com/Instant-Pot-Pressure-Steamer-Sterilizer/dp/B08PQ2KWHS
No price element found for https://www.walmart.com/ip/Instant-Pot-Pro-6-Quart-Electric-Pressure-Cooker-10-in-1-Slow-Rice-Grain-Steamer-Saut-Sous-Vide-Yogurt-Maker-Sterilizer-Warmer/674041054?from=/search
No price element found for https://www.bestbuy.com/site/instant-pot-6qt-pro-electric-pressure-cooker-black/6452

Only Amazon

In [ ]:
import requests
from bs4 import BeautifulSoup
import mysql.connector

# MySQL configuration
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'RootPass@#007',
    'database': 'price_tracker_db'
}

# Connect to MySQL
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()


cursor.execute(create_table_query)

# List of Amazon product URLs
amazon_urls = [
    'https://www.amazon.com/New-Apple-AirPods-Max-Space/dp/B08PZHYWJS',
    'https://www.amazon.com/ring-doorbell/s?k=ring+doorbell',
    'https://www.amazon.com/Instant-Pot-Pressure-Steamer-Sterilizer/dp/B08PQ2KWHS',
    'https://www.amazon.com/Dyson-V8-Absolute-Cordless-Vacuum/dp/B01EX9D3UW',
    'https://www.amazon.com/Meta-Quest-128GB-Breakthrough-Reality-3/dp/B0C8VKH1ZH'
]

# Function to scrape product data and store it in the database
def scrape_and_store_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract product name and price based on the website structure
    product_name_element = soup.find(id = 'productTitle')
    price_element = soup.find(id='priceblock_ourprice')

    if product_name_element and price_element:
        product_name = product_name_element.get_text().strip()
        price = float(price_element.get_text().replace('$', '').replace(',', '').strip())

        # Insert data into the database
        insert_query = "INSERT INTO products (product_name, price, source_url) VALUES (%s, %s, %s)"
        cursor.execute(insert_query, (product_name, price, url))
        conn.commit()

# Run scraping and storing data for Amazon URLs
for amazon_url in amazon_urls:
    scrape_and_store_data(amazon_url)

# Close the database connection
conn.close()


In [1]:
%pip install plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 63.0 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
@app.route("/")
def index():
    return render_template("index.html")

@app.route("/airpods.html")
def airpods():
    return render_template("airpods.html")

@app.route("/security_cam.html")
def security_cam():
    return render_template("security_cam.html")

@app.route("/electric_cooker.html")
def electric_cooker():
    return render_template("electric_cooker.html")

@app.route("/meta_quest_3.html")
def meta_quest_3():
    return render_template("meta_quest_3.html")

@app.route("/vaccum.html")
def vaccum():
    return render_template("vaccum.html")



In [ ]:
# from flask import Flask, render_template, request, jsonify
# import mysql.connector

# app = Flask(__name__)
# # app = Flask(__name__, static_url_path='/static')

# # MySQL Configuration
# db = mysql.connector.connect(
#     host="localhost",
#     user="root",
#     password="RootPass@#007",
#     database="price_tracker_db"
# )
# cursor = db.cursor()

# # Define the Product Price class to represent price history
# class ProductPrice:
#     def __init__(self, website, price, date):
#         self.website = website
#         self.price = price
#         self.date = date

# # Route to the main page
# @app.route("/")
# def index():
#     return render_template("index.html")

# # API endpoint to fetch price data
# @app.route("/get_prices", methods=["GET"])
# def get_prices():
#     product_id = request.args.get("product_id")

#     # Fetch price data from the database
#     cursor.execute("SELECT website, price, date FROM price_history WHERE product_id = %s", (product_id,))
#     rows = cursor.fetchall()

#     price_data = {}
#     for row in rows:
#         website, price, date = row
#         if website not in price_data:
#             price_data[website] = []
#         price_data[website].append({"price": price, "date": date})

#     return jsonify(price_data)

# if __name__ == "__main__":
#     app.run(debug=True)






from flask import Flask, render_template
import mysql.connector
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

app = Flask(__name__)

# MySQL Configuration
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="RootPass@#007",
    database="price_tracker_db"
)
cursor = db.cursor()

# Function to fetch price data from the database
def fetch_price_data(product_id):
    cursor.execute("SELECT website, price, date FROM price_history WHERE product_id = %s", (product_id,))
    rows = cursor.fetchall()

    price_data = {}
    for row in rows:
        website, price, date = row
        if website not in price_data:
            price_data[website] = []
        price_data[website].append({"price": price, "date": date})

    return price_data

# Function to fetch the current price
def fetch_current_price(product_id):
    cursor.execute("SELECT price FROM products WHERE id = %s", (product_id,))
    current_price = cursor.fetchone()[0]
    return current_price

# Function to generate the line graph
def generate_line_graph(price_data):
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    for website, data in price_data.items():
        prices = [entry["price"] for entry in data]
        dates = [entry["date"] for entry in data]
        
        fig.add_trace(go.Scatter(x=dates, y=prices, mode='lines+markers', name=website))

    fig.update_layout(title_text='Price History Over Time',
                      xaxis_title='Date',
                      yaxis_title='Price (USD)')

    return fig

@app.route("/")
def index():
    return render_template("index.html")

@app.route("/airpods.html")
def airpods():
    return render_template("airpods.html")

@app.route("/security_cam.html")
def security_cam():
    return render_template("security_cam.html")

@app.route("/electric_cooker.html")
def electric_cooker():
    return render_template("electric_cooker.html")

@app.route("/meta_quest_3.html")
def meta_quest_3():
    return render_template("meta_quest_3.html")

@app.route("/vaccum.html")
def vaccum():
    return render_template("vaccum.html")



# Route to product details page with line graph
@app.route("/product/<product_id>")
def product_details(product_id):
    # Fetch product details from the database
    cursor.execute("SELECT product_name FROM products WHERE id = %s", (product_id,))
    product_name = cursor.fetchone()[0]

    # Fetch current price
    current_price = fetch_current_price(product_id)

    # Fetch price data from the database
    price_data = fetch_price_data(product_id)

    # Generate line graph
    line_graph = generate_line_graph(price_data)

    # Convert the Plotly figure to HTML
    line_graph_html = line_graph.to_html(full_html=False)

    return render_template("product_details.html", product_name=product_name, current_price=current_price, line_graph=line_graph_html)

if __name__ == "__main__":
    app.run(debug=True)


















In [1]:
product_name = 'AirPods Max'

product_id_mapping = {
            'AirPods Max': 1,
            'Ring Video Doorbell': 2,
            'Instant Pot Pro': 3,
            'Dyson V8 Absolute': 5,
            'Meta Quest 3': 4,
        }

product_id_mapping[product_name]

1

In [19]:
import mysql.connector

# MySQL Configuration
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'RootPass@#007',
    'database': 'price_tracker_db'
}

# Function to fetch scrape_date from the database
# def fetch_scrape_date(product_id):
#     try:
#         with mysql.connector.connect(**db_config) as connection:
#             cursor = connection.cursor(dictionary=True)
#             query = "SELECT scrape_date FROM products WHERE p_id = %s"
#             params = (product_id,)
#             cursor.execute(query, params)
#             scrape_date = cursor.fetchone()
#             return scrape_date['scrape_date'] if scrape_date else None
#     except mysql.connector.Error as err:
#         print(f"Error fetching scrape_date: {err}")
#         return None

# Function to fetch the lowest price
# def fetch_current_price(product_id):
#     try:
#         with mysql.connector.connect(**db_config) as connection:
#             cursor = connection.cursor()
#             cursor.execute("SELECT MIN(price) AS min_price FROM products WHERE p_id = %s", (product_id,))
#             min_price = cursor.fetchone()[0]
#             return min_price
#     except mysql.connector.Error as err:
#         print(f"Error: {err}")
#         return None

def fetch_timestamp_price_data(product_id):
    try:
        with mysql.connector.connect(**db_config) as connection:
            cursor = connection.cursor(dictionary=True)
            query = "SELECT DATE_FORMAT(scrape_date, '%Y-%m-%d %H:%i:%s') AS timestamp, price FROM products WHERE p_id = %s"
            params = (product_id,)
            print("Executing query:", query, "with parameters:", params)
            cursor.execute(query, params)
            data = cursor.fetchall()
            print("Fetched data:", data)  # Add this line to print fetched data
            return data
    except mysql.connector.Error as err:
        print(f"Error fetching timestamp and price data: {err}")
        return None

# Example usage:
product_id = 1  # Replace with the actual product ID
# scrape_date = fetch_scrape_date(product_id)
# print(f"Scrape Date: {scrape_date}")

timestamp_price_data = fetch_timestamp_price_data(product_id)
print(f"Timestamp and Price Data: {timestamp_price_data}")


Executing query: SELECT DATE_FORMAT(scrape_date, '%Y-%m-%d %H:%i:%s') AS timestamp, price FROM products WHERE p_id = %s with parameters: (1,)
Error fetching timestamp and price data: Not enough parameters for the SQL statement
Timestamp and Price Data: None


In [23]:
def fetch_timestamp_price_data(product_id):
    try:
        with mysql.connector.connect(**db_config) as connection:
            cursor = connection.cursor(dictionary=True)
            query = "SELECT DATE_FORMAT(scrape_date, '%Y-%m-%d %H:%i:%s') AS timestamp, price FROM products WHERE p_id = %(product_id)s"
            params = {'product_id': product_id}
            # print("Executing query:", query, "with parameters:", params)
            cursor.execute(query, params)
            data = cursor.fetchall()
            print("Fetched data:", data)
            return data
    except mysql.connector.Error as err:
        print(f"Error fetching timestamp and price data: {err}")
        return None

# Example usage:
product_id = 1  # Replace with the actual product ID

timestamp_price_data = fetch_timestamp_price_data(product_id)
print(f"Timestamp and Price Data: {timestamp_price_data}")


Fetched data: [{'timestamp': '2023-11-12 22:35:01', 'price': 479.99}, {'timestamp': '2023-11-12 22:35:01', 'price': 549.0}, {'timestamp': '2023-11-12 22:35:03', 'price': 479.99}, {'timestamp': '2023-11-13 12:45:25', 'price': 479.99}, {'timestamp': '2023-11-13 12:45:26', 'price': 549.0}, {'timestamp': '2023-11-13 12:45:27', 'price': 479.99}, {'timestamp': '2023-11-13 20:29:03', 'price': 479.99}, {'timestamp': '2023-11-13 20:29:04', 'price': 549.0}, {'timestamp': '2023-11-13 20:29:05', 'price': 479.99}, {'timestamp': '2023-11-13 20:35:13', 'price': 479.99}, {'timestamp': '2023-11-13 20:35:14', 'price': 549.0}, {'timestamp': '2023-11-13 20:35:16', 'price': 479.99}, {'timestamp': '2023-11-13 20:59:26', 'price': 479.99}, {'timestamp': '2023-11-13 20:59:26', 'price': 549.0}, {'timestamp': '2023-11-13 20:59:28', 'price': 479.99}, {'timestamp': '2023-11-13 21:20:58', 'price': 479.99}, {'timestamp': '2023-11-13 21:20:59', 'price': 549.0}, {'timestamp': '2023-11-13 21:21:00', 'price': 479.99}, {